In [57]:
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error
import os
import pandas as pd
from pathlib import Path
import numpy as np

def MASE(A, F, y_train):
    return mean_absolute_scaled_error(A, F, y_train=y_train)


def SMAPE(a, f, _=None):
    return 1 / len(a) * np.sum(2 * np.abs(f - a) / (np.abs(a) + np.abs(f)) * 100)

def calculate_metrics(path, fedot_type = False):
    metrics_f = {}
    df_dict = {}
    for i in os.listdir(path):
        if fedot_type:
            if not i.__contains__('log') or not i.__contains__('report'):
                try:
                    df = pd.read_csv(Path(path, i,'prediction.csv'),index_col=0)
                    df_dict.update({i:df})
                    metrics_f[i] = SMAPE(df['label'], df['prediction'])
                except Exception:
                    _= 1
        else:
            if i.endswith('csv'):
                idx = i.split('_')[0]
                ids_to_proceed.append(idx)
                df = pd.read_csv(Path(path, i))
                df_dict.update({idx:df})
                targets[idx] = df['value']
                metrics_f[idx] = SMAPE(df['value'], df['predict'])
    return metrics_f, df_dict

In [70]:
from fedot_ind.api.utils.path_lib import PROJECT_PATH
autogluon = PROJECT_PATH + '/benchmark/results/benchmark_results/autogluon'
nbeats = PROJECT_PATH+'/benchmark/results/benchmark_results/nbeats'
fedot_ssa = PROJECT_PATH+'/benchmark/results/ts_uni_forecasting/ssa_results'
fedot_ar = PROJECT_PATH+'/benchmark/results/ts_uni_forecasting/ar'
ids_to_proceed = []
global_metrics = {}
targets= {}

In [71]:
global_metrics['fedot_ssa'], df_fedot = calculate_metrics(fedot_ssa, True)
global_metrics['fedot_ar'], df_fedot_ar = calculate_metrics(fedot_ar, True)

In [72]:
global_metrics['autogluon'], df_ag = calculate_metrics(autogluon)
global_metrics['nbeats'], df_nb = calculate_metrics(nbeats)

In [ ]:
for ds in df_fedot.keys():
    df_fedot[ds]['AG_pred'] = df_ag[ds]['predict']
    df_fedot[ds]['NB_pred'] = df_nb[ds]['predict']
    df_fedot[ds].plot()

In [98]:
df = []
for k, v in global_metrics.items():
    df.append(pd.DataFrame(v, index=[k]))

In [101]:
df_full = pd.concat([*df]).T.fillna(10000)
df_full['Difference_fedot_ar'] = (df_full.iloc[:, 2:3].min(axis=1) - df_full['fedot_ar'])
df_full['Difference_fedot_ssa'] = (df_full.iloc[:, 2:3].min(axis=1) - df_full['fedot_ssa'])
df_full['Fedot_Wins'] = df_full.apply(lambda row: 'Win' if row.loc['Difference_fedot_ssa']>0 or row.loc['Difference_fedot_ar']>0 else 'Loose', axis=1)
df_full[df_full['Fedot_Wins'] == 'Win']

,fedot_ssa,fedot_ar,autogluon,nbeats,Difference_fedot_ar,Difference_fedot_ssa,Fedot_Wins
D1002,5.941966,4.567342,4.962077,4.996679,0.394734,-0.979889,Win
D1019,1.600375,1.921184,1.603820,1.575932,-0.317364,0.003445,Win
D1032,12.227498,9.308087,9.377229,4.481296,0.069143,-2.850268,Win
D1104,2.525388,3.662158,2.560476,1.533204,-1.101682,0.035088,Win
D1124,0.911771,10000.000000,2.715447,2.442675,-9997.284553,1.803676,Win
...,...,...,...,...,...,...,...
Y16315,24.760798,10000.000000,44.938315,40.060936,-9955.061685,20.177517,Win
Y20781,31.651655,9.713997,10.500872,19.325849,0.786875,-21.150783,Win
Y3814,5.252030,3.712337,3.859581,5.472598,0.147244,-1.392449,Win
Y5212,9.654241,3.366590,10.957446,1.831877,7.590856,1.303205,Win


In [106]:
percent = (df_full[df_full['Fedot_Wins'] == 'Win'].shape[0]/df_full.shape[0])*100
print(f'Percentage of Fedot Wins among all models - {round(percent)} %')

Percentage of Fedot Wins among all models - 44 %


In [63]:
for i in ['D', 'W',  'M', 'Q', 'Y', 'all']:
    quantile_df = pd.DataFrame()
    if i != 'all':
        filter_col = [col for col in df_full if col.startswith(i)]
        d__ = df_full[filter_col]
    else:
        d__ = df_full
    quantiles = [0.5]
    d__ = d__.dropna(axis=1)
    for q in quantiles:

        d_ = d__.quantile(q, axis=1).round(3)
        quantile_df = pd.concat([quantile_df, d_], axis=1)
        if np.isnan(d_[0]):
            continue

        #quantile_df.to_excel(f'{i}_quantiles.xlsx', index=True)
        print(i)
        print(f'{q} quantile')
        print('metric')
        for j, v in enumerate(d_.sort_values().index):
            print(j + 1, v, round(d_[v], 2))

    quantile_df.columns = quantiles

D
0.5 quantile
metric
1 nbeats 1.73
2 autogluon 2.25
3 fedot_ar 3.11
4 fedot_ssa 3.37
W
0.5 quantile
metric
1 nbeats 5.26
2 autogluon 6.76
3 fedot_ar 9.5
4 fedot_ssa 10.53
M
0.5 quantile
metric
1 nbeats 7.21
2 autogluon 7.3
3 fedot_ssa 10.91
4 fedot_ar 11.45
Q
0.5 quantile
metric
1 autogluon 6.12
2 nbeats 9.29
3 fedot_ar 10.41
4 fedot_ssa 11.46
all
0.5 quantile
metric
1 nbeats 4.36
2 autogluon 4.56
3 fedot_ar 6.31
4 fedot_ssa 7.98


In [64]:
top_k = 1
wins_count = d__.apply(lambda x: x.nsmallest(top_k)).notnull().sum(axis=1).sort_values(ascending=False)

# Кол-во побед

In [65]:
wins_count

nbeats       245
autogluon    224
fedot_ar      98
dtype: int64

In [66]:
df = df_full.T

In [67]:
df

,fedot_ssa,fedot_ar,autogluon,nbeats
D1002,5.941966,4.567342,4.962077,4.996679
D1019,1.600375,1.921184,1.603820,1.575932
D1032,12.227498,9.308087,9.377229,4.481296
D1091,3.366175,2.093836,1.524169,0.952591
D1101,3.931032,3.072753,2.308593,2.325867
...,...,...,...,...
W53,8.076001,8.362080,6.735541,8.469682
W56,15.420568,13.462251,6.644513,2.787917
W61,5.644613,3.831377,2.832076,3.246926
W63,3.680823,2.477384,1.978506,1.430815


In [68]:
df['Difference'] = (df.iloc[:, 2:3].min(axis=1) - df['fedot_ar'])

In [69]:
df

,fedot_ssa,fedot_ar,autogluon,nbeats,Difference
D1002,5.941966,4.567342,4.962077,4.996679,0.394734
D1019,1.600375,1.921184,1.603820,1.575932,-0.317364
D1032,12.227498,9.308087,9.377229,4.481296,0.069143
D1091,3.366175,2.093836,1.524169,0.952591,-0.569667
D1101,3.931032,3.072753,2.308593,2.325867,-0.764161
...,...,...,...,...,...
W53,8.076001,8.362080,6.735541,8.469682,-1.626539
W56,15.420568,13.462251,6.644513,2.787917,-6.817738
W61,5.644613,3.831377,2.832076,3.246926,-0.999301
W63,3.680823,2.477384,1.978506,1.430815,-0.498878


In [15]:
df_full.to_csv(PROJECT_PATH + '/benchmark/results/m4_results.csv')

In [99]:
daily_results = PROJECT_PATH+'/benchmark/results/ts_uni_forecasting/metrics_report.csv'

In [106]:
plots = os.listdir(PROJECT_PATH+'/benchmark/results/ts_uni_forecasting/')

In [ ]:
for ds in plots:
    try:
        _ = pd.read_csv(PROJECT_PATH+'/benchmark/results/ts_uni_forecasting/'+ds+'/prediction.csv')
        del _['Unnamed: 0']
        _.plot()
    except Exception as eee:
        print(eee)

In [100]:
df_daily = pd.read_csv(daily_results)
df_daily = df_daily.fillna(0)
df_daily = df_daily[df_daily['Fedot_Industrial']!=0]

In [101]:
df_daily['Differnce'] = (df_daily.iloc[:, 1:3].min(axis=1) - df_daily['Fedot_Industrial'])

In [102]:
df_daily

,Unnamed: 0,autogluon,nbeats,Fedot_Industrial,Differnce
400,Q10070,0.966082,3.156489,1.706921,-0.740839
401,Q10262,6.085463,6.143364,8.487427,-2.401964
402,Q10292,19.784539,14.367727,31.618978,-17.251250
403,Q10466,5.081976,5.695602,7.007306,-1.925330
404,Q10598,3.815163,3.568740,5.225793,-1.657053
405,Q10665,13.816496,38.893301,67.860367,-54.043872


In [88]:
df_daily['Fedot_Wins'] = df_daily['Differnce'].apply(lambda x: 'Win' if x >0 else 'Loose')

In [89]:
df_daily[df_daily['Fedot_Wins'] == 'Win']

,Unnamed: 0,autogluon,nbeats,Fedot_Industrial,Differnce,Fedot_Wins
0,D1002,4.962077,4.996679,4.920500,0.041577,Win
1,D1019,1.603820,1.575932,1.318095,0.257837,Win
5,D1104,2.560476,1.533204,1.261557,0.271647,Win
7,D1162,1.855330,2.169810,1.559203,0.296127,Win
15,D1317,3.284125,3.058453,3.028002,0.030451,Win
...,...,...,...,...,...,...
377,M5684,7.602714,4.998233,3.876020,1.122212,Win
383,M6225,1.262179,1.165949,1.144854,0.021094,Win
390,M690,19.297647,18.324071,17.525603,0.798468,Win
395,M8497,53.243833,156.956136,10.505216,42.738617,Win


In [90]:
clear_csv = df_daily.iloc[:, :-2].rank(axis=1)

In [91]:
clear_csv[clear_csv['autogluon'] == 1].shape[0]

126

In [92]:
clear_csv[clear_csv['nbeats'] == 1].shape[0]

175

In [93]:
clear_csv[clear_csv['Fedot_Industrial'] == 1].shape[0]

99

In [94]:
pd.DataFrame(clear_csv.rank(axis=1).mean(axis=0).sort_values(), columns=['mean rank'])

,mean rank
nbeats,1.78
autogluon,1.97
Fedot_Industrial,2.25
